In [1]:
import pandas as pd

# Load the CSV file
file_path = 'drive data.csv'
data = pd.read_csv(file_path)

# Combine columns into descriptive prompts
def create_prompt(row):
    return (f"During the {row['Day/Night']} time, the weather was {row['Weather Conditions']} "
            f"with a temperature of {row['Temperature[°C]']}°C, "
            f"humidity at {row['Humidity[%]']}%, and precipitation of {row['Precipitation[mm]']} mm.")

data['prompt'] = data.apply(create_prompt, axis=1)
print(data[['prompt']].head())  # Preview the prompts


                                              prompt
0  During the Day time, the weather was Clear sky...
1  During the Day time, the weather was Overcast ...
2  During the Night time, the weather was Partly ...
3  During the Night time, the weather was Partly ...


In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load BART model and tokenizer
model_name = "facebook/bart-large"  # Standard BART model
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


c:\Users\fatem\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\fatem\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fatem\.cache\huggingface\hub\models--facebook--bart-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admi

In [3]:
# Generate text for each prompt
generated_texts = []

for prompt in data['prompt']:
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1, temperature=0.7)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_texts.append(generated_text)

# Add generated texts back to the DataFrame
data['generated_text'] = generated_texts

# Save to a new CSV
output_path = 'Bart_result.csv'
data.to_csv(output_path, index=False)
print(f"Generated descriptions saved to {output_path}")


c:\Users\fatem\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated descriptions saved to Bart_result.csv
